## Use opence-v1.4.1

In [14]:
#  # mount drive
# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)
# root_dir = "/content/gdrive/MyDrive"
# # change the OS to use your project folder as the working directory
# notebooks = "/post_ocr_repository/post_ocr_correction/notebooks/en"
# import os
# os.chdir(root_dir + notebooks)

In [15]:
# # HAL
# file_dir = '/home/jnaiman/wwt_image_extraction/alignments/'
# output_folder = '/home/jnaiman/data/morgan/'
# # utils from local
# from sys import path
# path.append('/home/jnaiman/libraries/')
# from utils_ocr_mini import align_texts_fast

# local
file_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/alignments/'
output_folder = '/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/'
from sys import path
path.append('../scienceDigitization/text_mining_ocr_and_pdf/ocr_spell_check/')
from importlib import reload
import utils
reload(utils)
from utils import align_texts_fast, get_fill_in_types

realign_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/ocr_pdf_aligned_sents_each_realign5/'


ender = '_small_words_pageLevel'


window_length = 100 # for subsetting the data

npages_train = 1000
npages_val = 250
npages_test = 250

In [16]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
import numpy as np

#from sys import path
#path.append('/home/jnaiman/.local')

from nltk.lm import Vocabulary
#import sys
#sys.path.append("../../lib")
#from metrics import levenshtein
import pickle
from glob import glob

from Levenshtein import distance
import re
from tqdm.auto import tqdm
import Levenshtein

import time
#from multiprocessing import Pool
from multiprocess import Pool # not sure if this is needed only on the mac?


# from importlib import reload
# import utils_ocr_mini
# reload(utils_ocr_mini)
# from utils_ocr_mini import align_texts_fast

In [17]:
def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]

def levenshtein(reference, hypothesis, progress_bar = False):
    assert len(reference) == len(hypothesis)
    text = zip(reference, hypothesis)
    if progress_bar:
        text = tqdm(text, total = len(reference))
    d = [distance(r, h) for r, h in text]
    output = pd.DataFrame({"reference":reference, "hypothesis":hypothesis})\
    .assign(distance = lambda df: d)\
    .assign(
        cer = lambda df: df.apply(
            lambda r: 100 * r["distance"] / max(len(r["reference"]), 1), 
            axis = 1
        )
    )
    return output

Get all data, format maybe:

In [18]:
align_files = glob(realign_dir + '*.pickle')
len(align_files)

7857

Load training data:

In [19]:
def make_dataframe(npages, arts_save = [], pages_save = [], imod=1000):
    # arts_save = []
    # pages_save = []
    train_pdf = []; train_ocr = []
    pages_save_out = []; filenames = []
    pdf_clean = []; ocr_clean = []
    while len(train_pdf) < npages:
        if len(train_pdf)%imod == 0:
            print('on', len(train_pdf), 'of', npages)
        i = np.random.randint(len(align_files))
        with open(align_files[i],'rb') as f:
            article = pickle.load(f)
        page_keys = list(article.keys())
        if len(page_keys) > 0:
            j = np.random.randint(len(page_keys))
            # check already in there
            alreadyIn = False
            if align_files[i].split('/')[-1].split('.pickle')[0] in arts_save:
                ind = arts_save.index(align_files[i].split('/')[-1].split('.pickle')[0])
                if page_keys[j] in pages_save: # also in there, correct one?
                    if pages_save[ind] == page_keys[j]: # same index
                        alreadyIn = True
            if not alreadyIn:
                arts_save.append(align_files[i].split('/')[-1].split('.pickle')[0])
                pages_save.append(page_keys[j])
                if article[page_keys[j]] != {}:
                    train_pdf.append(article[page_keys[j]]['PDF'])
                    # for formatting
                    train_ocr.append(article[page_keys[j]]['OCR'].replace('^','@'))
                    # also cleaned
                    pdf_clean.append(article[page_keys[j]]['PDF'].replace('@',''))
                    ocr_clean.append(article[page_keys[j]]['OCR'].replace('^',''))
                    pages_save_out.append(page_keys[j])
                    filenames.append(align_files[i].split('/')[-1].split('.pickle')[0])

    train_df = pd.DataFrame({'gt':train_pdf, 'ocr':train_ocr, 'filename':filenames, 
                             'page':pages_save_out,
                            'gt_clean':pdf_clean, 'ocr_clean':ocr_clean})
    return train_df, arts_save, pages_save

In [20]:
train_data, arts_save, pages_save = make_dataframe(npages_train,
                                                   arts_save = [], pages_save = [])

on 0 of 1000


In [21]:
val_data, arts_save, pages_save = make_dataframe(npages_val, 
                                                arts_save=arts_save.copy(),
                                               pages_save=pages_save.copy())

on 0 of 250
on 0 of 250


In [22]:
test_data, arts_save, pages_save = make_dataframe(npages_test, 
                                                arts_save=arts_save.copy(),
                                               pages_save=pages_save.copy())

on 0 of 250
on 0 of 250
on 0 of 250
on 0 of 250
on 0 of 250


Combine all together to for vocabulary:

In [23]:
data = pd.concat([train_data,val_data,test_data],ignore_index=True)

In [24]:
data.head()

,gt,ocr,filename,page,gt_clean,ocr_clean
0,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...,1111_1111.5432d_hatp13_psfilefixedRTM_hocr,page8,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...
1,". Finally, the accurate, observational determi...",@ Finally. the accurate. observational determi...,1009_1009.1978d_ms_psfilefixedRTM_hocr,page1,". Finally, the accurate, observational determi...",Finally. the accurate. observational determin...
2,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...,1007_1007.0493d_ms_psfilefixedRTM_hocr,page1,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...
3,where with are the density fractions of each m...,where with are the deusity fractions of each m...,0810_0810.0555d_11077aph_psfilefixedRTM_hocr,page4,where with are the density fractions of each m...,where with are the deusity fractions of each m...
4,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...,1106_1106.1432d_ms_psfilefixedRTM_hocr,page1,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...


In [25]:
# take out any weirdos
mask = (pd.isnull(data['gt'])) | (pd.isnull(data['ocr']))

In [26]:
len(data[mask])

0

In [27]:
data = data[~mask]

In [28]:
data.head()

,gt,ocr,filename,page,gt_clean,ocr_clean
0,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...,1111_1111.5432d_hatp13_psfilefixedRTM_hocr,page8,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...
1,". Finally, the accurate, observational determi...",@ Finally. the accurate. observational determi...,1009_1009.1978d_ms_psfilefixedRTM_hocr,page1,". Finally, the accurate, observational determi...",Finally. the accurate. observational determin...
2,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...,1007_1007.0493d_ms_psfilefixedRTM_hocr,page1,galaxies. BEGs are known to have positive colo...,galaxies. BECs are known to have positive colo...
3,where with are the density fractions of each m...,where with are the deusity fractions of each m...,0810_0810.0555d_11077aph_psfilefixedRTM_hocr,page4,where with are the density fractions of each m...,where with are the deusity fractions of each m...
4,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...,1106_1106.1432d_ms_psfilefixedRTM_hocr,page1,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...


Rename with equivalents for the original OCR method:

In [29]:
# train = train.rename(columns={"sentences source": "ocr_aligned", 
#                         "sentences target": "gs_aligned"})

data = data.rename(columns={'ocr_clean':"ocr_to_input", 
                            'ocr': "ocr_aligned",
                            'gt':"gs_aligned"})

In [30]:
data[['ocr_to_input','ocr_aligned', 'gs_aligned']].head()

,ocr_to_input,ocr_aligned,gs_aligned
0,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...,The HAT-P-13 system is unusual in that a trans...
1,Finally. the accurate. observational determin...,@ Finally. the accurate. observational determi...,". Finally, the accurate, observational determi..."
2,galaxies. BECs are known to have positive colo...,galaxies. BECs are known to have positive colo...,galaxies. BEGs are known to have positive colo...
3,where with are the deusity fractions of each m...,where with are the deusity fractions of each m...,where with are the density fractions of each m...
4,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...,resolution EVLA observations. We use a concord...


In [31]:
data[['ocr_to_input','ocr_aligned', 'gs_aligned']].applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,1500.000000,1500.000000,1500.000000
mean,3014.155333,3041.941333,3041.941333
std,1585.452724,1600.632865,1600.632865
min,1.000000,1.000000,1.000000
25%,1872.250000,1892.750000,1892.750000
50%,2866.500000,2897.500000,2897.500000
75%,4131.250000,4177.500000,4177.500000
max,7114.000000,8625.000000,8625.000000


In [32]:
levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
            hypothesis = data.ocr_to_input).cer.describe()

count    1500.000000
mean        5.413870
std         6.446605
min         0.000000
25%         2.402198
50%         4.319378
75%         6.750639
max        88.208617
Name: cer, dtype: float64

Create vocabulary:

In [33]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    1500.000000
mean        5.235680
std         5.824194
min         0.000000
25%         2.370930
50%         4.249685
75%         6.629447
max        83.165217
Name: cer, dtype: float64

In [34]:
# let's try with just test dataset:
#data = pd.concat([train_data,val_data,test_data],ignore_index=True)
levenshtein(reference = data.iloc[-len(test_data):].gs_aligned.replace("@",""), 
            hypothesis = data.iloc[-len(test_data):].ocr_aligned.replace("@","")).cer.describe()

count    250.000000
mean       6.048617
std        8.439131
min        0.000000
25%        2.599628
50%        4.564981
75%        7.002886
max       83.165217
Name: cer, dtype: float64

In [35]:
test_data

,gt,ocr,filename,page,gt_clean,ocr_clean
0,Define A@:= Since a complement space to the im...,Define AA:= Since a complement space to the im...,1112_1112.2544d_1112.2544_psfilefixedRTM_hocr,page6,Define A:= Since a complement space to the ima...,Define AA:= Since a complement space to the im...
1,In van den Bergh (2009) it was speculated that...,In van den Bereh (2009) it was speculated that...,0907_0907.3715d_0907.3715_psfilefixedRTM_hocr,page5,In van den Bergh (2009) it was speculated that...,In van den Bereh (2009) it was speculated that...
2,noise due to the low number density of galaxie...,noise due to the low number density of galaxie...,1003_1003.4211d_boosting_psfilefixedRTM_hocr,page11,noise due to the low number density of galaxie...,noise due to the low number density of galaxie...
3,X-ray observations with the observatory have e...,X-ray observations with the observatory have e...,0709_0709.1336d_paper_rev3_aph_psfilefixedRTM_...,page0,X-ray observations with the observatory have e...,X-ray observations with the observatory have e...
4,"strong O band, however. Methane shows only one...",strong @ band. however. Methane shows only one...,0310_astro-ph0310805d_saumon_psfilefixedRTM_hocr,page3,"strong O band, however. Methane shows only one...",strong band. however. Methane shows only one ...
...,...,...,...,...,...,...
245,"motions one can estimate, e.g., the expected m...",motions one can estimate. e.g.. the expected m...,1011_1011.4816d_15641_psfilefixedRTM_hocr,page6,"motions one can estimate, e.g., the expected m...",motions one can estimate. e.g.. the expected m...
246,"numerous in our star sample (15381 stars), it ...",numerous in our star sample (15381 stars). it ...,0512_astro-ph0512352d_ngc7789_exp_psfilefixedR...,page7,"numerous in our star sample (15381 stars), it ...",numerous in our star sample (15381 stars). it ...
247,we summarize our main conclusions. We assume t...,we summarize our main conclusions. We assume t...,1008_1008.2986d_nsc_psfilefixedRTM_hocr,page2,we summarize our main conclusions. We assume t...,we summarize our main conclusions. We assume t...
248,RVM sign convention problem The phase of maxim...,RVM sign convention problem The phase of maxim...,1111_1111.4270d_FCamiloJ2030+3641_psfilefixedR...,page4,RVM sign convention problem The phase of maxim...,RVM sign convention problem The phase of maxim...


In [36]:
rerun_vocab = False

if rerun_vocab:
    start_time = time.time()
    vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
    print(len(vocabulary))
    with open(output_folder+"data/vocabulary_new_pages_new"+ender+".pkl", "wb") as file:
        pickle.dump(vocabulary, file)
    end_time = time.time()
    print('time delta =', end_time-start_time, 'seconds, or', 
          (end_time-start_time)/60., 'minutes, or',
         (end_time-start_time)/60./60., 'hours')

408
time delta = 1.479722023010254 seconds, or 0.024662033716837565 minutes, or 0.0004110338952806261 hours


In [37]:
def mask_rename(datain):
    mask = (pd.isnull(datain['gt'])) | (pd.isnull(datain['ocr']))
    # datain = datain.rename(columns={'words source':"ocr_to_input", 
    #                             'words source aligned': "ocr_aligned",
    #                             'words target aligned':"gs_aligned"})
    datain = datain.rename(columns={'ocr_clean':"ocr_to_input", 
                            'ocr': "ocr_aligned",
                            'gt':"gs_aligned"})
    return datain

In [38]:
# train_data.to_pickle(output_folder+"/data/train_new_pages"+ender+".pkl")
# train_data.shape

Format and save training data:

In [39]:
train_data = mask_rename(train_data)

In [40]:
start_time = time.time()
with Pool(4) as p:
    train_aligned = list(p.imap_unordered(create_windows, 
                                          tqdm(zip(train_data.ocr_aligned, 
                                                   train_data.gs_aligned, 
                                                   [window_length for x in train_data.ocr_aligned]), 
                                               total = len(train_data.ocr_aligned)),
                                          chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()
end_time = time.time()
print('total time =', end_time-start_time, 'seconds, or', (end_time-start_time)/60., 'minutes')

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

(3041135, 2)
total time = 7.2342798709869385 seconds, or 0.12057133118311564 minutes


In [41]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,In this section we discuss the effects of clus...,In this section we discuss the effects of clus...
1,n this section we discuss the effects of clust...,n this section we discuss the effects of clust...
2,this section we discuss the effects of cluste...,this section we discuss the effects of cluste...
3,this section we discuss the effects of cluster...,this section we discuss the effects of cluster...
4,his section we discuss the effects of cluster ...,his section we discuss the effects of cluster ...


In [42]:
train_aligned.to_pickle(output_folder+"data/train_aligned_new_pages"+ender+".pkl")

For dev:

In [43]:
val_data = mask_rename(val_data)

In [44]:
dev_aligned = val_data.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(766032, 2)


,source,target
0,total contribution of these two populations re...,total contribution of these two populations re...
1,otal contribution of these two populations res...,otal contribution of these two populations res...
2,tal contribution of these two populations resu...,tal contribution of these two populations resu...
3,al contribution of these two populations resul...,al contribution of these two populations resul...
4,l contribution of these two populations result...,l contribution of these two populations result...


In [45]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,total contribution of these two populations re...,total contribution of these two populations re...
1,otal contribution of these two populations res...,otal contribution of these two populations res...
2,tal contribution of these two populations resu...,tal contribution of these two populations resu...
3,al contribution of these two populations resul...,al contribution of these two populations resul...
4,l contribution of these two populations result...,l contribution of these two populations result...


In [46]:
dev_aligned.to_pickle(output_folder+"data/dev_aligned_new_pages"+ender+".pkl")

# ------------ HERE ---------------

In [16]:
# rerun_vocab = False

# if rerun_vocab:
#     start_time = time.time()
#     #vocabulary = Vocabulary(data[~mask].ocr_aligned.sum() + data[~mask].gs_aligned.sum())
#     vocabulary = Vocabulary(data[~mask]['words source'].sum() + data[~mask]['words target'].sum())
#     print(len(vocabulary))
#     with open(output_folder+"data/vocabulary_new_pages_new"+ender+".pkl", "wb") as file:
#         pickle.dump(vocabulary, file)
#     end_time = time.time()
#     print('time delta =', end_time-start_time, 'seconds, or', 
#           (end_time-start_time)/60., 'minutes, or',
#          (end_time-start_time)/60./60., 'hours')
#     # last run:
#     #time delta = 11285.80526137352 seconds, 188.09675435622532 minutes, ~3.3 hours on HAL

In [17]:
#output_folder+"data/vocabulary_new_pages_new"+ender+".pkl"

## Create windowed data

Read in only the training data and do stuff with that:

In [18]:
#train = pd.read_csv(file_dir + train_file)

# train = train.rename(columns={"sentences source": "ocr_aligned", 
#                         "sentences target": "gs_aligned"})

#mask = (pd.isnull(train['ocr_aligned'])) | (pd.isnull(train['gs_aligned']))
#mask = (pd.isnull(train['sentences source'])) | (pd.isnull(train['sentences target']))
mask = (pd.isnull(train_data['words source'])) | (pd.isnull(train_data['words target']))

train_data = train_data[~mask]

# now align
gs_aligned = []; ocr_aligned = []
gs = []; ocr = []
for o,p in zip(train_data['words source'].values, train_data['words target'].values):
    eops = Levenshtein.editops(o, p)
    ocr_text_aligned, pdf_text_aligned = align_texts_fast(o,p,eops)
    gs_aligned.append(pdf_text_aligned)
    # futze with ocr aligned so it matches data input
    ocr_text_aligned = ocr_text_aligned.replace('^', '@')
    ocr_aligned.append(ocr_text_aligned)
    gs.append(pdf_text_aligned.replace('@',''))
    ocr.append(ocr_text_aligned.replace('@',''))
    
train_data['gs_aligned'] = gs_aligned
train_data['ocr_aligned'] = ocr_aligned
train_data['gs'] = gs
train_data['ocr'] = ocr

In [19]:
mask = (pd.isnull(val_data['words source'])) | (pd.isnull(val_data['words target']))

val_data = val_data[~mask]

# now align
gs_aligned = []; ocr_aligned = []
gs = []; ocr = []
for o,p in zip(val_data['words source'].values, val_data['words target'].values):
    eops = Levenshtein.editops(o, p)
    ocr_text_aligned, pdf_text_aligned = align_texts_fast(o,p,eops)
    gs_aligned.append(pdf_text_aligned)
    # futze with ocr aligned so it matches data input
    ocr_text_aligned = ocr_text_aligned.replace('^', '@')
    ocr_aligned.append(ocr_text_aligned)
    gs.append(pdf_text_aligned.replace('@',''))
    ocr.append(ocr_text_aligned.replace('@',''))
    
val_data['gs_aligned'] = gs_aligned
val_data['ocr_aligned'] = ocr_aligned
val_data['gs'] = gs
val_data['ocr'] = ocr

In [20]:
mask = (pd.isnull(test_data['words source'])) | (pd.isnull(test_data['words target']))

test_data = test_data[~mask]

# now align
gs_aligned = []; ocr_aligned = []
gs = []; ocr = []
for o,p in zip(test_data['words source'].values, test_data['words target'].values):
    eops = Levenshtein.editops(o, p)
    ocr_text_aligned, pdf_text_aligned = align_texts_fast(o,p,eops)
    gs_aligned.append(pdf_text_aligned)
    # futze with ocr aligned so it matches data input
    ocr_text_aligned = ocr_text_aligned.replace('^', '@')
    ocr_aligned.append(ocr_text_aligned)
    gs.append(pdf_text_aligned.replace('@',''))
    ocr.append(ocr_text_aligned.replace('@',''))
    
test_data['gs_aligned'] = gs_aligned
test_data['ocr_aligned'] = ocr_aligned
test_data['gs'] = gs
test_data['ocr'] = ocr

In [24]:
# data = pd.concat([train_data,val_data,test_data],ignore_index=True)

# data['ocr'].iloc[:10].sum() + data['gs'].iloc[:10].sum()

In [25]:
rerun_vocab = False

if rerun_vocab:
    data = pd.concat([train_data,val_data,test_data],ignore_index=True)
    start_time = time.time()
    #vocabulary = Vocabulary(data[~mask].ocr_aligned.sum() + data[~mask].gs_aligned.sum())
    #vocabulary = Vocabulary(data[~mask]['ocr'].sum() + data[~mask]['gs'].sum())
    vocabulary = Vocabulary(data['ocr'].sum() + data['gs'].sum() + '@')
    print(len(vocabulary))
    with open(output_folder+"data/vocabulary_new_pages_new"+ender+".pkl", "wb") as file:
        pickle.dump(vocabulary, file)
    end_time = time.time()
    print('time delta =', end_time-start_time, 'seconds, or', 
          (end_time-start_time)/60., 'minutes, or',
         (end_time-start_time)/60./60., 'hours')
    # last run:
    #time delta = 11285.80526137352 seconds, 188.09675435622532 minutes, ~3.3 hours on HAL

496
time delta = 145.02771306037903 seconds, or 2.417128551006317 minutes, or 0.040285475850105286 hours


In [143]:
#p = Pool(4)

In [26]:
#train = data.drop(dev.index)
train_data.to_pickle(output_folder+"/data/train_new_pages"+ender+".pkl")
train_data.shape

(100000, 16)

In [27]:
# df = train#.head(100)
# train_aligned = list(p.imap_unordered(create_windows, 
#                                       tqdm(zip(df.ocr_aligned, 
#                                                df.gs_aligned, 
#                                                [window_length for x in df.ocr_aligned]), 
#                                            total = len(df.ocr_aligned)),
#                                       chunksize = 128))
# s = []
# for r in tqdm(train_aligned):
#     s.extend(r)
# train_aligned = pd.DataFrame(s, columns = ["source", "target"])
# print(train_aligned.shape)
# train_aligned.head()
start_time = time.time()
with Pool(4) as p:
    train_aligned = list(p.imap_unordered(create_windows, 
                                          tqdm(zip(train_data.ocr_aligned, 
                                                   train_data.gs_aligned, 
                                                   [window_length for x in train_data.ocr_aligned]), 
                                               total = len(train_data.ocr_aligned)),
                                          chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()
end_time = time.time()
print('total time =', end_time-start_time, 'seconds, or', (end_time-start_time)/60., 'minutes')

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

(13826088, 2)
total time = 27.231825828552246 seconds, or 0.4538637638092041 minutes


In [28]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,This research was supported by the TIER 11th ...,This research was supported by the TIFR 11th ...
1,This research was supported by the TIER 11th F...,This research was supported by the TIFR 11th F...
2,his research was supported by the TIER 11th Fi...,his research was supported by the TIFR 11th Fi...
3,is research was supported by the TIER 11th Fiv...,is research was supported by the TIFR 11th Fiv...
4,s research was supported by the TIER 11th Five...,s research was supported by the TIFR 11th Five...


Same thing for validation set:

In [29]:
# valid = pd.read_csv(file_dir + val_file)

# mask = (pd.isnull(valid['sentences source'])) | (pd.isnull(valid['sentences target']))

# valid = valid[~mask]

# # now align
# gs_aligned = []; ocr_aligned = []
# for o,p in zip(valid['sentences source'].values, valid['sentences target'].values):
#     eops = Levenshtein.editops(o, p)
#     ocr_text_aligned, pdf_text_aligned = align_texts_fast(o,p,eops)
#     gs_aligned.append(pdf_text_aligned)
#     # futze with ocr aligned so it matches data input
#     ocr_text_aligned = ocr_text_aligned.replace('^', '@')
#     ocr_aligned.append(ocr_text_aligned)
    
# valid['gs_aligned'] = gs_aligned
# valid['ocr_aligned'] = ocr_aligned

In [30]:
dev_aligned = val_data.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(690924, 2)


,source,target
0,First. claiming both uses for the spectroscop...,"First, claiming both uses for the spectroscop..."
1,First. claiming both uses for the spectroscopi...,"First, claiming both uses for the spectroscopi..."
2,irst. claiming both uses for the spectroscopic...,"irst, claiming both uses for the spectroscopic..."
3,rst. claiming both uses for the spectroscopic ...,"rst, claiming both uses for the spectroscopic ..."
4,st. claiming both uses for the spectroscopic s...,"st, claiming both uses for the spectroscopic s..."


In [31]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,First. claiming both uses for the spectroscop...,"First, claiming both uses for the spectroscop..."
1,First. claiming both uses for the spectroscopi...,"First, claiming both uses for the spectroscopi..."
2,irst. claiming both uses for the spectroscopic...,"irst, claiming both uses for the spectroscopic..."
3,rst. claiming both uses for the spectroscopic ...,"rst, claiming both uses for the spectroscopic ..."
4,st. claiming both uses for the spectroscopic s...,"st, claiming both uses for the spectroscopic s..."


Finally save:

In [32]:
train_aligned.to_pickle(output_folder+"data/train_aligned_new_pages"+ender+".pkl")
dev_aligned.to_pickle(output_folder+"data/dev_aligned_new_pages"+ender+".pkl")

In [33]:
output_folder+"data/train_aligned_new_pages"+ender+".pkl"

'/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/data/train_aligned_new_pages_small_words.pkl'

In [33]:
output_folder+"data/dev_aligned_new_pages"+ender+".pkl"

'/Users/jnaiman/Downloads/tmp/ocrpost/data/morgan/data/dev_aligned_new_pages_small_words.pkl'

In [34]:
# for i in range(len(train_aligned)):
#     d = train_aligned.iloc[i]
#     if '@' in d['source'] or '@' in d['target']:
#         import sys; sys.exit()

In [35]:
#d['source'], d['target']

# --- orig stuff ---

In [2]:
new_data_dir = '/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/'

In [3]:
output_folder = '/home/mcosi153/post_ocr_correction/data/en'

In [4]:
new_data_files = sorted(os.listdir(new_data_dir))
len(new_data_files)

7892

In [6]:
new_data_files = glob(new_data_dir+"*.pickle")

In [7]:
new_data_files[:3]

['/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/1005_1005.2297d_14607_psfilefixedRTM_hocr.pickle',
 '/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/9811_astro-ph9811382d_buzzoni_psfilefixedRTM_hocr.pickle',
 '/home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/0908_0908.4092d_Jenkins09b_psfilefixedRTM_hocr.pickle']

In [8]:
len(new_data_files)

7891

In [9]:
file0 = 1
file_name = new_data_files[file0]

In [10]:

!pip install lxml


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [11]:
# with open(file_name,'rb') as f:
#     ffout,pdf_text_pages,ocr_text_pages,alignment_pages,pdf_sents_aligned_pages,ocr_sents_aligned_pages,pdf_sents_pages,ocr_sents_pages = pickle.load(f)

In [12]:
with open(file_name,'rb') as f:
    realign_pages = pickle.load(f)


In [13]:
len(realign_pages)

9

In [14]:
#len(pdf_sents_aligned_pages)

In [15]:
#files = sorted(os.listdir(folder))
#len(files)
files = sorted(os.listdir(new_data_dir))
len(files)
files[:3]

['.ipynb_checkpoints',
 '0001_astro-ph0001013d_letter_psfilefixedRTM_hocr - Copy.pickle',
 '0001_astro-ph0001013d_letter_psfilefixedRTM_hocr.pickle']

In [16]:
import glob

files = glob.glob(new_data_dir + '/**/*.pickle', recursive=True)
len(files)

7891

In [17]:
from multiprocessing import Pool

# def extract(name):
   # with open(name) as file: #this is used to open and read text files
        #return file.readlines()


def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
    
p = Pool(4)
    
# #data = list(p.imap_unordered(extract, tqdm(files), chunksize = 128,))
# #len(data)
# #data = extract('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle')

source_dataframe = pd.DataFrame({"ocr_aligned":[],"gs_aligned":[]})

#for file_name in files:
#for file_index in range(len(files)):# can change length of files ex: files/10 or 1000 to find out if there is less time
import random
#r = list(range(4773))
r = list(range(40))
random.shuffle(r)
pdf_ocr = []
for file_index in r: 
    file_name = files[file_index]
    if file_index % 200 == 0:
        print(str(file_index) + " " + file_name)
    for key in realign_pages:
        if 'PDF' in realign_pages[key] and 'OCR' in realign_pages[key]:
            pdf_ocr.append({'ocr_aligned': realign_pages[key]['OCR'],'gs_aligned': realign_pages[key]['PDF']})

    source_dataframe = pd.DataFrame(pdf_ocr)
    #fh = open(str(file_name), 'rb')
    #ffout,pdf_text_pages, ocr_text_pages, alignment_pages, pdf_sents_aligned_pages, ocr_sents_aligned_pages, pdf_sents_pages, ocr_sents_pages = pickle.load(fh)
    ##print(len(pdf_sents_aligned_pages.keys()))
    ##import sys; sys.exit()
    #for key in pdf_sents_aligned_pages.keys():
      ##for ocr_sents, pdf_sents in zip(ocr_sents_aligned_pages[key], pdf_sents_aligned_pages[key]):
        #if ocr_sents_aligned_pages[key] != "" and pdf_sents_aligned_pages[key] != "":
        ##print("not empty")
        ##print(key)
            #new_dataframe = pd.DataFrame({"ocr_aligned":ocr_sents_aligned_pages[key], "gs_aligned":pdf_sents_aligned_pages[key]})
            #source_dataframe = pd.concat([source_dataframe, new_dataframe], ignore_index = True, sort = False)
#         else:
#             #print("empty string")
#             print(key)

# #use source_dataframe add code here to remove str from list
# #empty_list = []
#print(source_dataframe.shape)

# # for index in range(len(source_dataframe.ocr_aligned)-1):
# #   if type(source_dataframe.ocr_aligned[index]) != list:
# #     source_dataframe.ocr_aligned.pop(index)
# #     source_dataframe.gs_aligned.pop(index)

# #print(source_dataframe.shape)

0 /home/mcosi153/post_ocr_correction/data/ocr_pdf_aligned_sents_each_realign4/1005_1005.2297d_14607_psfilefixedRTM_hocr.pickle


In [18]:
source_dataframe.head()

,ocr_aligned,gs_aligned
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...
2,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
3,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
4,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...


In [19]:
# for index_number in range(len(source_dataframe.gs_aligned)):
#     if index_number % 5000 == 0:
#       #print(source_dataframe.ocr_aligned[index_number])
#       #print(source_dataframe.gs_aligned[index_number])

In [54]:
#source_dataframe.to_csv("full_source_dataframe_to_split.csv")

In [19]:
# the rest of the code is using "data" for dataframe
data = source_dataframe 
#data = pd.read_csv("full_train_data_672699.csv" )

In [20]:
#check how many rows in data
print(data.shape)

(120, 2)


In [21]:
data.head()

,ocr_aligned,gs_aligned
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...
2,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
3,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
4,À basic step when asseniblung a vunthetec stel...,A basic step when assem@bling a synthetic stel...


In [38]:
#data = data.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

In [25]:
#this is what i added to make a list with one pickle file
#lst = pd.read_pickle('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle')
#with open('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle','rb') as f:
# ffout,pdf_text_pages,ocr_text_pages,alignment_pages,pdf_sents_aligned_pages,ocr_sents_aligned_pages,pdf_sents_pages,ocr_sents_pages = pickle.load(f)

In [26]:
#df1 = pd.read_pickle('../../../../OCR Spell check WWT Independent Study/data/ocr_pdf_aligned_sents_each2/0311_astro-ph0311556d_belsol_psfilefixedRTM_hocr.pickle')

In [27]:
#print(df1.shape)

In [28]:

# data = pd.DataFrame(data, 
#                     columns = ["ocr_to_input", 
#                                "ocr_aligned", 
#                                "gs_aligned"])\
# .assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
#         ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
#         gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

# print(data.shape)
# data.head()



In [29]:
#data.to_csv("./github_training_data.csv")

In [22]:
data.applymap(len).describe()

,ocr_aligned,gs_aligned
count,120.000000,120.000000
mean,77.333333,77.333333
std,0.946762,0.946762
min,76.000000,76.000000
25%,76.000000,76.000000
50%,78.000000,78.000000
75%,78.000000,78.000000
max,78.000000,78.000000


In [23]:
!pip install Levenshtein
import pandas as pd
from Levenshtein import distance
import re
from tqdm.auto import tqdm
       
def levenshtein(reference, hypothesis, progress_bar = False):
    assert len(reference) == len(hypothesis)
    text = zip(reference, hypothesis)
    if progress_bar:
        text = tqdm(text, total = len(reference))
    d = [distance(r, h) for r, h in text]
    output = pd.DataFrame({"reference":reference, "hypothesis":hypothesis})\
    .assign(distance = lambda df: d)\
    .assign(
        cer = lambda df: df.apply(
            lambda r: 100 * r["distance"] / max(len(r["reference"]), 1), 
            axis = 1
        )
    )
    return output

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [41]:
# levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
#             hypothesis = data.ocr_to_input).cer.describe()

In [24]:
levenshtein(reference = data.gs_aligned, 
            hypothesis = data.ocr_aligned).cer.describe()

count    120.000000
mean       6.848853
std        4.233466
min        1.315789
25%        1.315789
50%        7.692308
75%       11.538462
max       11.538462
Name: cer, dtype: float64

In [105]:
vocabulary = Vocabulary(data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))
with open(output_folder+"/"+"data/vocabulary_new_pages1.pkl", "wb") as file:
    pickle.dump(vocabulary, file)

34


In [35]:
# data.gs_aligned[51173]

In [36]:
#output_folder = Path("../../data/en")

In [37]:
# data.ocr_aligned.pop(51173)
# data.gs_aligned.pop(51173)

In [38]:
# type(data.ocr_aligned[51172])

In [39]:
# vocab1 = []
# vocab2 = []

# for index in range(len(data.ocr_aligned)-1):
#   if type(data.ocr_aligned[index]) != type(vocab1):
#     print(data.ocr_aligned[index])

#   else:
#       vocab1 = vocab1 + data.ocr_aligned[index]
#     #source_dataframe.ocr_aligned.pop()
#     #source_dataframe.gs_aligned.pop()

# for index in range(len(data.gs_aligned)-1):
#   if type(data.gs_aligned[index]) != type(vocab2):
#     print(data.gs_aligned[index])
#   else:
#       vocab2 = vocab2 + data.gs_aligned[index]
#     #source_dataframe.ocr_aligned.pop()

# vocab = vocab1 + vocab2




In [40]:
# data.gs_aligned.sum() 

In [41]:
# for d in data.ocr_aligned.values:
#   if type(d) != list:
#     print(d, "here")

In [11]:
data.gs_aligned.values [- 1]

'@e confront the two models from Table @n this wavelength interval with each other. '

In [54]:
ocr = data.ocr_aligned.sum()
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:38


In [55]:
gs = data.gs_aligned.sum()
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:39


In [56]:
#vocabulary = Vocabulary(data.ocr_aligned.sum() + data.gs_aligned.sum())
vocab = ocr + gs
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:40


In [57]:
vocabulary = Vocabulary(ocr + gs)
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:41


In [58]:
# vocabulary = Vocabulary(vocab)
print(len(vocabulary))
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)


34
Current Time = 15:55:42


In [59]:
with open(output_folder+"/data/vocabulary_new_pages.pkl", "wb") as file:
    pickle.dump(vocabulary, file)
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 15:55:46


In [25]:
dev = data.sample(n = 5, random_state = 1)
dev.to_pickle(output_folder+"/data/dev_new_pages.pkl")
dev.shape

(5, 2)

In [26]:
train = data.drop(dev.index)
train.to_pickle(output_folder+"/data/train_new_pages.pkl")
train.shape

(115, 2)

In [27]:
train.applymap(len).describe()

,ocr_aligned,gs_aligned
count,115.000000,115.000000
mean,77.356522,77.356522
std,0.938376,0.938376
min,76.000000,76.000000
25%,76.000000,76.000000
50%,78.000000,78.000000
75%,78.000000,78.000000
max,78.000000,78.000000


In [28]:
dev.applymap(len).describe()

,ocr_aligned,gs_aligned
count,5.000000,5.000000
mean,76.800000,76.800000
std,1.095445,1.095445
min,76.000000,76.000000
25%,76.000000,76.000000
50%,76.000000,76.000000
75%,78.000000,78.000000
max,78.000000,78.000000


In [29]:
# levenshtein(reference = dev.gs_aligned.str.replace("@", ""), 
#              hypothesis = dev.ocr_to_input).cer.describe()

In [30]:
levenshtein(reference = dev.gs_aligned, 
            hypothesis = dev.ocr_aligned).cer.describe()

count    5.000000
mean     3.866397
std      3.492563
min      1.315789
25%      1.315789
50%      1.315789
75%      7.692308
max      7.692308
Name: cer, dtype: float64

In [31]:
window_length = 100

In [32]:
from multiprocessing import Pool

# def extract(name):
   # with open(name) as file: #this is used to open and read text files
        #return file.readlines()


def create_windows(x):
    A, B, window_length = x
    assert len(A) == len(B)
    return [(A[i:i + window_length], B[i:i + window_length]) 
            for i in range(len(A) + 1)]
p = Pool(4)

In [33]:
df = train#.head(100)
train_aligned = list(p.imap_unordered(create_windows, 
                                      tqdm(zip(df.ocr_aligned, 
                                               df.gs_aligned, 
                                               [window_length for x in df.ocr_aligned]), 
                                           total = len(df.ocr_aligned)),
                                      chunksize = 128))
s = []
for r in tqdm(train_aligned):
    s.extend(r)
train_aligned = pd.DataFrame(s, columns = ["source", "target"])
print(train_aligned.shape)
train_aligned.head()

  0%|          | 0/115 [00:00<?, ?it/s]

  0%|          | 0/115 [00:00<?, ?it/s]

(9011, 2)


,source,target
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,Pollax population svuthesis has extensively be...,tellar population synthesis has extensively be...
2,ollax population svuthesis has extensively bee...,ellar population synthesis has extensively bee...
3,llax population svuthesis has extensively been...,llar population synthesis has extensively been...
4,lax population svuthesis has extensively been ...,lar population synthesis has extensively been ...


In [34]:
train_aligned = train_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
train_aligned.head()

,source,target
0,MPollax population svuthesis has extensively b...,Stellar population synthesis has extensively b...
1,Pollax population svuthesis has extensively be...,tellar population synthesis has extensively be...
2,ollax population svuthesis has extensively bee...,ellar population synthesis has extensively bee...
3,llax population svuthesis has extensively been...,llar population synthesis has extensively been...
4,lax population svuthesis has extensively been ...,lar population synthesis has extensively been ...


In [35]:
dev_aligned = dev.apply(lambda r: create_windows((r["ocr_aligned"], r["gs_aligned"], window_length)), 
                            axis = 1).sum()
dev_aligned = pd.DataFrame(dev_aligned, columns = ["source", "target"])
print(dev_aligned.shape)
dev_aligned.head()

(389, 2)


,source,target
0,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
1,s a general feature. SSP post-MS evolution is ...,"s a general feature, SSP post-MS evolution is ..."
2,a general feature. SSP post-MS evolution is r...,"a general feature, SSP post-MS evolution is r..."
3,a general feature. SSP post-MS evolution is re...,"a general feature, SSP post-MS evolution is re..."
4,general feature. SSP post-MS evolution is rec...,"general feature, SSP post-MS evolution is rec..."


In [44]:
#trained_sample_df=train_aligned.head(1)

In [45]:
#trained_sample_df.to_csv("trained_sample.csv")

In [36]:
dev_aligned = dev_aligned.assign(source = lambda df: df.source.str.replace("@", ""))
dev_aligned.head()

,source,target
0,As a general feature. SSP post-MS evolution is...,"As a general feature, SSP post-MS evolution is..."
1,s a general feature. SSP post-MS evolution is ...,"s a general feature, SSP post-MS evolution is ..."
2,a general feature. SSP post-MS evolution is r...,"a general feature, SSP post-MS evolution is r..."
3,a general feature. SSP post-MS evolution is re...,"a general feature, SSP post-MS evolution is re..."
4,general feature. SSP post-MS evolution is rec...,"general feature, SSP post-MS evolution is rec..."


In [37]:
train_aligned.to_pickle(output_folder+"/data/train_aligned_new_pages.pkl")
dev_aligned.to_pickle(output_folder+"/data/dev_aligned_new_pages.pkl")

In [14]:
import numpy as np

In [18]:
df = train_aligned

In [19]:
len(df)

91919976

In [16]:
train_aligned_chunks = np.array_split(df, 92)
for i, df in enumerate(train_aligned_chunks):
    if i % 10 == 0 and i != 0:
        print(f"{i} dataframes have been created so far")

10 dataframes have been created so far
20 dataframes have been created so far
30 dataframes have been created so far
40 dataframes have been created so far
50 dataframes have been created so far
60 dataframes have been created so far
70 dataframes have been created so far
80 dataframes have been created so far
90 dataframes have been created so far


In [17]:
for i in range(92):
    train_aligned_chunks[i].to_pickle(output_folder+"/data/train_aligned_new_full"+str(i).zfill(4)+".pkl")

In [21]:
dev_df = dev_aligned
len(dev_df)

635

In [22]:
dev_df.to_pickle(output_folder+"/data/dev_aligned_new_full.pkl")

In [ ]:
train_aligned.to_pickle(output_folder+"/data/train_aligned_new_full.pkl")
dev_aligned.to_pickle(output_folder+"/data/dev_aligned_new_full.pkl")

In [ ]:
train = pd.read_pickle(output_folder+"/data/train_aligned_new_full.pkl")
train.shape

In [4]:
import pandas as pd

In [25]:
import numpy as np

In [26]:
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

In [27]:
df


,A,B,C,D
0,29,94,33,50
1,14,54,75,51
2,35,36,61,66
3,25,9,72,41
4,26,66,9,22
...,...,...,...,...
95,41,27,88,65
96,6,53,39,52
97,85,39,59,92
98,69,51,1,87


In [33]:
df_chunks = np.array_split(df, 10)

In [34]:
df_chunks[0]

,A,B,C,D
0,29,94,33,50
1,14,54,75,51
2,35,36,61,66
3,25,9,72,41
4,26,66,9,22
5,90,64,78,94
6,77,95,99,98
7,34,32,88,43
8,2,80,38,18
9,68,94,10,9


In [35]:
for i in range(10):
    df_chunks[i].to_pickle(output_folder+"/data/train_aligned_new_full"+str(i).zfill(3)+".pkl")

IndexError: list index out of range